In [1]:
import ast
from functools import cache
import inspect 
import time

In [2]:
def function_to_ast_string(f):
    # Get the AST in tree form
    node = ast.parse(inspect.getsource(f))
    
    # Convert to string. We can use a string here because the
    # function 
    return ast.dump(node)

def define_once_first_try(f):
    if f.__name__ in globals().keys():
        return globals()[f.__name__]
    
    return f

def define_once(f):
    if f.__name__ in globals().keys():
        if function_to_ast_string(f) == function_to_ast_string(globals()[f.__name__]):
            return globals()[f.__name__]
    
    return f

In [3]:
def time_function(f, *args):
    # We implement this function because we are sleeping, therefore
    # we want to measure wall time. We don't care about the function
    # output here, we just want to time the function for demonstration
    # purposes.
    start = time.time()
    result = f(*args)
    end = time.time()
    
    return end - start, result

def print_time(f, *args):
    elapsed, result = time_function(f, *args)
    print(f"{f.__name__} took {elapsed:.6f} secs with result {result}")

In [4]:
# Run this cell over and over. You will see that the redefined function always takes
# the same amount of time, while the function defined once at first takes one second
# but then takes a miniscule amount of time every subsequent time the cell is run.
# 
# Note that if the kernel is restarted, then the defined once function takes a long
# time to run again. That particular problem is out of scope for this example: if one
# wants to have a persistent cache, then it is possible to extend this idea (or better
# ideas) using an on-disk cache.

@cache
def function_that_gets_redefined(a: int) -> int:
    time.sleep(1)
    return a

@define_once_first_try
@cache
def function_that_is_defined_once_first_try(a: int) -> int:
    time.sleep(1)
    
    # Try changing the return result here (say by adding 5) and see if
    # the result changes in the print out.
    return a

@define_once
@cache
def function_that_is_defined_once(a: int) -> int:
    time.sleep(1)
    return a + 5


print_time(function_that_gets_redefined, 1)
print_time(function_that_is_defined_once_first_try, 1)
print_time(function_that_is_defined_once, 1)

function_that_gets_redefined took 1.004959 secs with result 1
function_that_is_defined_once_first_try took 1.003071 secs with result 1
function_that_is_defined_once took 1.001997 secs with result 6
